In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaTokenizer
import torchvision.models as models

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Adobe_InterIIT/image_video_embed.csv')
data.head()

<ipython-input-40-2523f48a2abc>:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/Adobe_InterIIT/image_video_embed.csv')


,index,id,date,likes,content,username,media,inferred company,photo_link,video_link,min_resolution_url,video_embed,image_embed
0,0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,['https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?...,[],[],NaN,"[[0.0218505859375, -0.023529052734375, -0.0206..."
1,1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,['https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?...,[],[],NaN,"[[0.07379150390625, 0.050384521484375, 0.02098..."
2,2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,['https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?...,[],[],NaN,"[[0.033538818359375, 0.0245208740234375, -0.07..."
3,3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,['https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?...,[],[],NaN,"[[-0.030914306640625, -0.0133056640625, -0.032..."
4,4,6,2020-11-15 16:01:08,525,An 85-year-old primary school in Shanghai has ...,cnni,[Video(thumbnailUrl='https://pbs.twimg.com/amp...,cnn,['https://pbs.twimg.com/amplify_video_thumb/13...,['https://video.twimg.com/amplify_video/132238...,https://video.twimg.com/amplify_video/13223822...,"[[-0.04480615630745888, 0.0036263801157474518,...",NaN


In [ ]:
data = data.iloc[:5]

In [ ]:
# Load RoBERTa model and tokenizer
roberta_model = RobertaModel.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Load ResNet50 model
resnet50_model = models.resnet50(pretrained=True)
resnet50_model.fc = nn.Sequential(nn.Linear(2048, 512), nn.ReLU())

# Define attention mechanism layers
class SelfAttention(nn.Module):
    def __init__(self, input_size):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(input_size, input_size)
        self.key = nn.Linear(input_size, input_size)
        self.value = nn.Linear(input_size, input_size)

    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        attn = torch.softmax(torch.matmul(q, k.transpose(-2, -1)), dim=-1)
        attended = torch.matmul(attn, v)
        return attended

class CrossAttentionModel(nn.Module):
    def __init__(self, roberta_model, resnet50_model, text_embedding_size, image_embedding_size):
        super(CrossAttentionModel, self).__init__()
        self.roberta = roberta_model
        self.resnet50 = resnet50_model
        self.text_embedding_size = text_embedding_size
        self.image_embedding_size = image_embedding_size

        # Define attention layers
        self.text_self_attention = SelfAttention(text_embedding_size)
        self.image_self_attention = SelfAttention(image_embedding_size)

        # Cross-attention weights
        self.attention_text_to_image = nn.Linear(text_embedding_size, image_embedding_size)
        self.attention_image_to_text = nn.Linear(image_embedding_size, text_embedding_size)

        self.regression = nn.Linear(512, 1)

    def forward(self, text_input_ids, image_input):
        # RoBERTa forward pass
        text_outputs = self.roberta(input_ids=text_input_ids)
        text_embeddings = text_outputs['last_hidden_state'][:, 0, :]  # Extract [CLS] token

        # ResNet50 forward pass
        image_embeddings = self.resnet50(image_input)
        image_embeddings = image_embeddings.view(image_embeddings.size(0), -1)

        # Self-attention for text and image
        attended_text = self.text_self_attention(text_embeddings)
        attended_image = self.image_self_attention(image_embeddings)

        # Cross-attention from text to image
        attended_text_to_image = self.attention_text_to_image(attended_text)
        attended_text_to_image = torch.matmul(torch.softmax(torch.matmul(attended_text_to_image, image_embeddings.T), dim=-1), image_embeddings)

        # Cross-attention from image to text
        attended_image_to_text = self.attention_image_to_text(attended_image)
        attended_image_to_text = torch.matmul(torch.softmax(torch.matmul(attended_image_to_text, text_embeddings.T), dim=-1), text_embeddings)

        outputs = self.regression(attended_text_to_image)

        return outputs
        ## return attended_text_to_image, attended_image_to_text

# Example usage
text_input_ids = tokenizer.encode("Your text here", return_tensors='pt')
image_input = torch.randn(1, 3, 224, 224)  # Example image input shape for ResNet50

model = CrossAttentionModel(roberta_model, resnet50_model, 768, 512)
attended_text, attended_image = model(text_input_ids, image_input)

In [6]:
attended_image.shape

torch.Size([1, 768])